<a href="https://colab.research.google.com/github/krish7101/100-days-of-code-python/blob/main/Prompt_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get the openai secret key
import getpass

secret_key = getpass.getpass('Please enter your openai key: ') # https://platform.openai.com/account/api-keys

Please enter your openai key: ··········


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.0 MB/s eta 0:00:00


In [ ]:
# Define two variants of the prompt
prompt_A = """Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

prompt_B = """Product description: A home milkshake maker.
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A watch that can tell accurate time in space.
Seed words: astronaut, space-hardened, eliptical orbit
Product names: AstroTime, SpaceGuard, Orbit-Accurate, EliptoTime.

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

test_prompts = [prompt_A, prompt_B]

import pandas as pd
import openai

openai.api_key = secret_key

def get_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message['content']

# Iterate through the prompts and get responses
test_prompts = [prompt_A, prompt_B]
responses = []
num_tests = 5

for idx, prompt in enumerate(test_prompts):
    # prompt number as a letter
    var_name = chr(ord('A') + idx)

    for i in range(num_tests):
        # Get a response from the model
        response = get_response(prompt)

        data = {
            "variant": var_name,
            "prompt": prompt,
            "response": response
            }
        responses.append(data)

# Convert responses into a DataFrame
df = pd.DataFrame(responses)

# Save the DataFrame as a CSV file
df.to_csv("responses.csv", index=False)

print(df)

  variant                                             prompt  \
0       A  Product description: A pair of shoes that can ...   
1       A  Product description: A pair of shoes that can ...   
2       A  Product description: A pair of shoes that can ...   
3       A  Product description: A pair of shoes that can ...   
4       A  Product description: A pair of shoes that can ...   
5       B  Product description: A home milkshake maker.\n...   
6       B  Product description: A home milkshake maker.\n...   
7       B  Product description: A home milkshake maker.\n...   
8       B  Product description: A home milkshake maker.\n...   
9       B  Product description: A home milkshake maker.\n...   

                                            response  
0  1. AdaptaShoe\n2. FitMorphs\n3. SizeFlex\n4. O...  
1  1. OmniSize Shoes\n2. FitAll Footwear\n3. Adap...  
2  1. Adapt-a-fit shoes\n2. AnySize shoes\n3. Omn...  
3  1. FlexiFit\n2. EasyFit\n3. OmniShoe\n4. Adapt...  
4  1. Adapt-A-Shoe\n

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# load the responses.csv file
df = pd.read_csv("responses.csv")

# Shuffle the DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# Assuming df is your DataFrame and 'response' is the column with the text you want to test
response_index = 0
df['feedback'] = pd.Series(dtype='str')  # add a new column to store feedback

def on_button_clicked(b):
    global response_index
    #  convert thumbs up / down to 1 / 0
    user_feedback = 1 if b.description == "👍" else 0

    # update the feedback column
    df.at[response_index, 'feedback'] = user_feedback

    response_index += 1
    if response_index < len(df):
        update_response()
    else:
        # save the feedback to a CSV file
        df.to_csv("results.csv", index=False)

        print("A/B testing completed. Here's the results:")
        # Calculate score for each variant and count the number of rows per variant
        summary_df = df.groupby('variant').agg(count=('feedback', 'count'), score=('feedback', 'mean')).reset_index()
        print(summary_df)

def update_response():
    new_response = df.iloc[response_index]['response']
    new_response = "<p>" + new_response + "</p>" if pd.notna(new_response) else "<p>No response</p>"
    response.value = new_response
    count_label.value = f"Response: {response_index + 1} / {len(df)}"

response = widgets.HTML()
count_label = widgets.Label()

update_response()

thumbs_up_button = widgets.Button(description='👍')
thumbs_up_button.on_click(on_button_clicked)

thumbs_down_button = widgets.Button(description='👎')
thumbs_down_button.on_click(on_button_clicked)

button_box = widgets.HBox([thumbs_down_button, thumbs_up_button])

display(response, button_box, count_label)

HTML(value='<p>Adapt-a-fit, OmniShoe, FitAll, FlexFit.</p>')

Label(value='Response: 1 / 10')

A/B testing completed. Here's the results:
  variant  count  score
0       A      5    0.0
1       B      5    1.0
